## Stock Market Comparison Analysis

In [1]:
import pandas as pd
import yfinance as yf
import plotly.io as pio
import plotly.graph_objects as go
pio.templates.default = "plotly_white"

In [2]:
# Define the tickers for Apple and Google
apple_ticker = 'AAPL'
google_ticker = 'GOOGL'
microsoft_ticker = 'MSFT'

# Define the date range for the last quarter
start_date = '2023-07-01'
end_date = '2023-09-30'

# Fetch historical stock price data using yfinance
apple_data = yf.download(apple_ticker, start=start_date, end=end_date)
google_data = yf.download(google_ticker, start=start_date, end=end_date)
microsoft_data = yf.download(microsoft_ticker, start=start_date, end=end_date)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [3]:
# Calculate daily returns
apple_data['Daily_Return'] = apple_data['Adj Close'].pct_change()
google_data['Daily_Return'] = google_data['Adj Close'].pct_change()
microsoft_data['Daily_Return'] = microsoft_data['Adj Close'].pct_change()

# Create a figure to visualize the daily returns
fig = go.Figure()

fig.add_trace(go.Scatter(x=apple_data.index, y=apple_data['Daily_Return'],
                         mode='lines', name='Apple', line=dict(color='blue')))
fig.add_trace(go.Scatter(x=google_data.index, y=google_data['Daily_Return'],
                         mode='lines', name='Google', line=dict(color='green')))
fig.add_trace(go.Scatter(x=microsoft_data.index, y=microsoft_data['Daily_Return'],
                         mode='lines', name='Microsoft', line=dict(color='red')))

fig.update_layout(title='Daily Returns for Apple, Google and Microsoft (Last Quarter)',
                  xaxis_title='Date', yaxis_title='Daily Return',
                  legend=dict(x=0.9, y=1.3))

fig.show()

In [4]:
# Calculate cumulative returns for the last quarter
apple_cumulative_return = (1 + apple_data['Daily_Return']).cumprod() - 1
google_cumulative_return = (1 + google_data['Daily_Return']).cumprod() - 1
microsoft_cumulative_return = (1 + microsoft_data['Daily_Return']).cumprod() - 1

# Create a figure to visualize the cumulative returns
fig = go.Figure()

fig.add_trace(go.Scatter(x=apple_cumulative_return.index, y=apple_cumulative_return,
                         mode='lines', name='Apple', line=dict(color='blue')))
fig.add_trace(go.Scatter(x=google_cumulative_return.index, y=google_cumulative_return,
                         mode='lines', name='Google', line=dict(color='green')))
fig.add_trace(go.Scatter(x=microsoft_cumulative_return.index, y=microsoft_cumulative_return,
                         mode='lines', name='Microsoft', line=dict(color='red')))

fig.update_layout(title='Cumulative Returns for Apple, Google and Microsoft (Last Quarter)',
                  xaxis_title='Date', yaxis_title='Cumulative Return',
                  legend=dict(x=0.9, y=1.3))

fig.show()

In [5]:
# Calculate historical volatility (standard deviation of daily returns)
apple_volatility = apple_data['Daily_Return'].std()
google_volatility = google_data['Daily_Return'].std()
microsoft_volatility = microsoft_data['Daily_Return'].std()

# Create a figure to compare volatility
fig1 = go.Figure()
fig1.add_bar(x=['Apple', 'Google', 'Microsoft'], y=[apple_volatility, google_volatility, microsoft_volatility],
             text=[f'{apple_volatility:.4f}', f'{google_volatility:.4f}', f'{microsoft_volatility:.4f}'],
             textposition='auto', marker=dict(color=['blue', 'green', 'red']))

fig1.update_layout(title='Volatility Comparison (Last Quarter)',
                   xaxis_title='Stock', yaxis_title='Volatility (Standard Deviation)',
                   bargap=0.5)
fig1.show()

""""
We first calculated the historical volatility for both Apple and Google stocks. Volatility is a measure of how much the stock’s price 
fluctuates over time. In this case, we are calculating the standard deviation of daily returns to measure the volatility. 
Then we visualized the calculated volatility to assess and compare the volatility or risk associated with both Apple and Google stocks 
during the specified period. We can see that Google’s volatility is higher than Apple’s.

It indicates that Google’s stock price experienced larger price fluctuations or greater price variability over the last quarter. 
Here’s what this difference in volatility may indicate:
1.Google’s stock is considered riskier compared to Apple. Investors generally associate higher volatility with higher risk because it 
implies that the stock price can change significantly in a short period.

2.Google’s stock may be more sensitive to market conditions, economic factors, or company-specific news and events. 
This heightened sensitivity can result in larger price swings.

3.Traders and investors with a higher risk tolerance might find Google’s stock appealing if they are looking for opportunities to 
profit from short-term price movements.
"""

'"\nWe first calculated the historical volatility for both Apple and Google stocks. Volatility is a measure of how much the stock’s price \nfluctuates over time. In this case, we are calculating the standard deviation of daily returns to measure the volatility. \nThen we visualized the calculated volatility to assess and compare the volatility or risk associated with both Apple and Google stocks \nduring the specified period. We can see that Google’s volatility is higher than Apple’s.\n\nIt indicates that Google’s stock price experienced larger price fluctuations or greater price variability over the last quarter. \nHere’s what this difference in volatility may indicate:\n1.Google’s stock is considered riskier compared to Apple. Investors generally associate higher volatility with higher risk because it \nimplies that the stock price can change significantly in a short period.\n\n2.Google’s stock may be more sensitive to market conditions, economic factors, or company-specific news and

In [6]:
market_data = yf.download('^GSPC', start=start_date, end=end_date)  # S&P 500 index as the market benchmark

# Calculate daily returns for both stocks and the market
apple_data['Daily_Return'] = apple_data['Adj Close'].pct_change()
google_data['Daily_Return'] = google_data['Adj Close'].pct_change()
microsoft_data['Daily_Return'] = microsoft_data['Adj Close'].pct_change()
market_data['Daily_Return'] = market_data['Adj Close'].pct_change()

# Calculate Beta for Apple and Google
var_market = market_data['Daily_Return'].var()

cov_apple = apple_data['Daily_Return'].cov(market_data['Daily_Return'])
beta_apple = cov_apple / var_market

cov_google = google_data['Daily_Return'].cov(market_data['Daily_Return'])
beta_google = cov_google / var_market

cov_microsoft = microsoft_data['Daily_Return'].cov(market_data['Daily_Return'])
beta_microsoft = cov_microsoft / var_market

# Compare Beta values
if beta_apple > beta_google and beta_apple > cov_microsoft:
    conclusion = "Apple is more volatile (higher Beta) compared to Google and Microsoft."
elif beta_google > beta_apple and beta_google > cov_microsoft:
    conclusion = "Google is more volatile (higher Beta) compared to Apple and Microsoft."
else:
    conclusion = "Miscrosoft is more volatile (higher Beta) compared to Apple and Google."

# Print the conclusion
print("Beta for Apple:", beta_apple)
print("Beta for Google:", beta_google)
print("Beta for Microsoft:", beta_microsoft)
print(conclusion)

[*********************100%%**********************]  1 of 1 completed

Beta for Apple: 1.2257459204729633
Beta for Google: 1.53030690283569
Beta for Microsoft: 1.3028061286331063
Google is more volatile (higher Beta) compared to Apple and Microsoft.
